In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/drive/MyDrive/final_project/CP_AC_url.csv")

In [3]:
df

,asin,imageURLHighRes
0,7506284774,https://images-na.ssl-images-amazon.com/images...
1,751182059X,NaN
2,7538425780,https://images-na.ssl-images-amazon.com/images...
3,7550001049,https://images-na.ssl-images-amazon.com/images...
4,8199208503,NaN
...,...,...
589351,B01HISEZ0W,https://images-na.ssl-images-amazon.com/images...
589352,B01HIT2M0Q,https://images-na.ssl-images-amazon.com/images...
589353,B01HIZEVKE,https://images-na.ssl-images-amazon.com/images...
589354,B01HJ6B1G4,https://images-na.ssl-images-amazon.com/images...


In [4]:
df.dropna(inplace=True)


In [5]:
df

,asin,imageURLHighRes
0,7506284774,https://images-na.ssl-images-amazon.com/images...
2,7538425780,https://images-na.ssl-images-amazon.com/images...
3,7550001049,https://images-na.ssl-images-amazon.com/images...
7,8199404531,https://images-na.ssl-images-amazon.com/images...
11,8288850723,https://images-na.ssl-images-amazon.com/images...
...,...,...
589351,B01HISEZ0W,https://images-na.ssl-images-amazon.com/images...
589352,B01HIT2M0Q,https://images-na.ssl-images-amazon.com/images...
589353,B01HIZEVKE,https://images-na.ssl-images-amazon.com/images...
589354,B01HJ6B1G4,https://images-na.ssl-images-amazon.com/images...


In [6]:
# df는 나누고자 하는 원본 DataFrame이라고 가정합니다.
num_rows = df.shape[0]  # DataFrame의 총 행 수를 구합니다.

# DataFrame을 두 부분으로 나눕니다.
df_first_half = df.iloc[:num_rows // 2]  # 첫 번째 절반
df_second_half = df.iloc[num_rows // 2:]  # 두 번째 절반

# 결과 확인
print(df_first_half)
print(df_second_half)


              asin                                    imageURLHighRes
0       7506284774  https://images-na.ssl-images-amazon.com/images...
2       7538425780  https://images-na.ssl-images-amazon.com/images...
3       7550001049  https://images-na.ssl-images-amazon.com/images...
7       8199404531  https://images-na.ssl-images-amazon.com/images...
11      8288850723  https://images-na.ssl-images-amazon.com/images...
...            ...                                                ...
297268  B00H9XEQTW  https://images-na.ssl-images-amazon.com/images...
297269  B00H9Y5Q2M  https://images-na.ssl-images-amazon.com/images...
297270  B00H9YAFA0  https://images-na.ssl-images-amazon.com/images...
297271  B00H9YAFC8  https://images-na.ssl-images-amazon.com/images...
297273  B00HA21SWU  https://images-na.ssl-images-amazon.com/images...

[225786 rows x 2 columns]
              asin                                    imageURLHighRes
297274  B00HA21V9K  https://images-na.ssl-images-amazon.com/ima

In [7]:
df = df_first_half

In [8]:
df

,asin,imageURLHighRes
0,7506284774,https://images-na.ssl-images-amazon.com/images...
2,7538425780,https://images-na.ssl-images-amazon.com/images...
3,7550001049,https://images-na.ssl-images-amazon.com/images...
7,8199404531,https://images-na.ssl-images-amazon.com/images...
11,8288850723,https://images-na.ssl-images-amazon.com/images...
...,...,...
297268,B00H9XEQTW,https://images-na.ssl-images-amazon.com/images...
297269,B00H9Y5Q2M,https://images-na.ssl-images-amazon.com/images...
297270,B00H9YAFA0,https://images-na.ssl-images-amazon.com/images...
297271,B00H9YAFC8,https://images-na.ssl-images-amazon.com/images...


In [3]:
df

NameError: name 'df' is not defined

#ㅁㄴㅇㅁㄴㅇ

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models
import requests
from io import BytesIO
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import numpy as np

# GPU를 사용할 수 있는지 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 이미지 전처리 함수 정의
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 이미지 처리 함수 정의
def process_image(image_url):
    try:
        # URL에서 이미지 읽기
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert('RGB')
        image = preprocess(image)
        return image.unsqueeze(0), image_url  # 배치 차원 추가 및 URL 반환
    except Exception as e:
        print(f"Error processing image from URL {image_url}: {e}")
        return None, image_url

# 이미지 일괄 처리 함수 정의
def process_images(image_urls):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(process_image, image_urls))

    images = [img for img, url in results if img is not None]
    successful_urls = [url for img, url in results if img is not None]

    if images:
        images = torch.cat(images, dim=0)
    else:
        images = torch.Tensor()  # 빈 텐서 반환

    return images, successful_urls

# 이미지 임베딩 추출 함수 정의
def extract_embeddings(images):
    model = models.resnet50(pretrained=True).to(device)
    model.eval()

    with torch.no_grad():
        images = images.to(device)  # GPU로 이동
        embeddings = model(images)

    return embeddings.cpu().numpy()  # CPU로 이동하여 반환

def add_embedding_column(df, batch_size=1000):
    df_copy = df.copy()  # 데이터프레임 복사
    image_urls = df_copy.loc[df_copy['imageURLHighRes'].str.endswith(('.jpg', '.png')), 'imageURLHighRes']

    embeddings_list = []
    successful_urls_list = []

    for i in range(0, len(image_urls), batch_size):
        batch_urls = image_urls[i:i+batch_size].tolist()
        images, successful_urls = process_images(batch_urls)
        successful_urls_list.extend(successful_urls)  # 성공한 URL 저장

        if images.numel() > 0:  # 이미지가 성공적으로 처리된 경우에만 임베딩 추출
            embeddings = extract_embeddings(images)
            embeddings_list.append(embeddings)

    # 성공한 URL에 대한 데이터프레임 필터링
    df_filtered = df_copy[df_copy['imageURLHighRes'].isin(successful_urls_list)]

    # 임베딩 데이터 생성
    if embeddings_list:
        embeddings_concat = np.concatenate(embeddings_list, axis=0)
        df_embeddings = pd.DataFrame({'embedding': embeddings_concat.tolist()}, index=df_filtered.index[:len(embeddings_concat)])
        df_filtered = pd.concat([df_filtered, df_embeddings], axis=1)

    return df_filtered

# 데이터프레임에 임베딩 열 추가
df_with_embedding = add_embedding_column(df)
print(df_with_embedding)

Using device: cuda
Error processing image from URL https://images-na.ssl-images-amazon.com/images/I/41aq5emPwbL.jpg: cannot identify image file <_io.BytesIO object at 0x7b4c564c9e90>


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 156MB/s]


Error processing image from URL https://images-na.ssl-images-amazon.com/images/I/418RR4tnYGL.jpg: cannot identify image file <_io.BytesIO object at 0x7b4c3d2cf3d0>
Error processing image from URL https://images-na.ssl-images-amazon.com/images/I/51w6HViBT-L.jpg: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error processing image from URL https://images-na.ssl-images-amazon.com/images/I/41VqYtzTYdL.jpg: cannot identify image file <_io.BytesIO object at 0x7b4c5013bce0>
Error processing image from URL https://images-na.ssl-images-amazon.com/images/I/41a7UM-98GL.jpg: cannot identify image file <_io.BytesIO object at 0x7b4c50156200>
Error processing image from URL https://images-na.ssl-images-amazon.com/images/I/51LlAPqCZwL.jpg: cannot identify image file <_io.BytesIO object at 0x7b4c50147060>
Error processing image from URL https://images-na.ssl-images-amazon.com/images/I/51x%2BIGP09OL.jpg: cannot identify image file <_io.BytesIO object at 0x7b4c3da6e430>
E

In [1]:
df_with_embedding = add_embedding_column(df)
print(df_with_embedding)

NameError: name 'add_embedding_column' is not defined

In [2]:
df_with_embedding.to_csv('/content/drive/MyDrive/final_project/ver1.csv', index=False)


NameError: name 'df_with_embedding' is not defined

In [ ]:
print(embedding)

tensor([[ 2.1390e-01, -3.5807e-01, -1.2483e+00, -1.7291e+00, -4.6118e-01,
          5.1551e-01, -1.7790e+00, -1.0150e+00, -1.2947e+00, -8.9464e-01,
          8.3902e-02, -1.1446e+00, -1.0852e+00, -6.5285e-01, -4.8678e-01,
         -8.5834e-01, -2.5381e-01, -1.2877e+00, -7.1870e-01, -7.3093e-01,
         -2.0958e+00, -1.4655e-02, -7.5978e-01, -9.3135e-01, -6.1664e-01,
         -6.8885e-01, -8.5323e-01, -5.2088e-02, -1.2425e+00, -1.1108e+00,
         -2.2812e+00, -1.2802e+00, -1.5407e+00, -1.0926e+00, -1.0945e-01,
         -2.1629e+00, -1.4275e+00, -1.9002e+00,  4.5889e-01, -3.7083e-01,
         -2.4195e+00, -1.3599e+00, -1.3267e+00, -1.0017e+00, -9.1577e-01,
         -1.2331e-01, -1.7237e+00, -8.2322e-01, -1.5344e+00, -2.1319e+00,
         -5.8228e-01,  2.3833e-02, -7.5643e-01, -1.5038e+00, -2.3084e+00,
         -1.8573e+00, -2.7406e+00, -2.8670e+00, -2.2699e+00, -4.0961e-01,
         -2.9475e+00, -1.3493e+00, -1.8658e+00,  1.3003e-02, -1.3223e+00,
         -5.5937e-01, -3.1153e-01, -8.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')